In [1]:
import numpy as np, pandas as pd 
pd.set_option('display.max_columns', None)
import sqlite3, sqlalchemy 

In [2]:
season = "2016_17" 
engine = sqlalchemy.create_engine("sqlite:///../../DB/ballbase.db") 

master = pd.read_sql("master_"+season, con=engine).fillna(0).drop("Tm", axis=1)
master.columns 

Index(['Player', 'Pos', 'Age', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '3P',
       '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA', 'FT%', 'ORB',
       'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '3PAr', 'AST%',
       'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM', 'ORB%', 'OWS',
       'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP', 'WS', 'WS/48',
       'DRtg', 'ORtg', 'And1', 'BadPass', 'Blkd', 'C%', 'LostBall', 'Off.',
       'Off..1', 'On-Off', 'OnCourt', 'PF%', 'PG%', 'PGA', 'SF%', 'SG%',
       'Shoot', 'Shoot.1', 'T2P', 'T2PA', 'T3PA', 'TAST', 'TBLK', 'TDRB',
       'TFG', 'TFGA', 'TFT', 'TFTA', 'TMP', 'TORB', 'TPF', 'TPTS', 'TSTL',
       'TTOV', 'TTRB'],
      dtype='object')

In [3]:
averages = master[['Player', 'Pos', 'Age', 'PTS', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'MP', 'FGA', 'FG%', '3PA', '3P%', 'FTA', 'FT%', 'PF']] 
totals = master.loc[:, ['Player', 'TPTS', 'TSTL',
       'TTOV', 'TTRB', 'T2P', 'T2PA', 'T3PA', 'TAST', 'TBLK', 'TDRB',
       'TFG', 'TFGA', 'TFT', 'TFTA', 'TMP', 'TORB', 'TPF']]
advanced = master.loc[:, ['Player', 'DRtg', 'ORtg',
       '3PAr', 'AST%', 'BLK%', 'BPM', 'DBPM', 'DRB%', 'DWS', 'FTr', 'OBPM',
       'ORB%', 'OWS', 'PER', 'STL%', 'TOV%', 'TRB%', 'TS%', 'USG%', 'VORP',
       'WS', 'WS/48']] 


contents = [averages, totals, advanced] 

In [4]:
# Create HTML table version 
for i in range(len(contents)):
    contents[i] = contents[i].to_html(table_id="table", classes = "table table-hover table-stripped table-bordered table-striped display nowrap", index=False)

# Current Date 
# from datetime import datetime
# curr_date = datetime.today().strftime('%Y-%m-%d %H:%M:%S') 
# <p style="text-align:center;font-size:10px;font-family:calibri;color:black;">Last Update: {curr_date} ET </p>
# Create page content 
for i in range(len(contents)):
    contents[i] = f"""
    {{%extends 'TableBase.html'%}}
    {{% block table %}}    
    {contents[i]}
    {{% endblock %}}
"""

In [5]:
def write_to_file(path, content): 
    file = open(path, "w", encoding="utf-8") 
    file.write(content) 
    file.close 


file_names=[f"Averages_{season}.html", f"Totals_{season}.html", f"Advanced_{season}.html"] 

for i in range(len(contents)): 
    path = f"../../templates/Tables/{file_names[i]}" 
    write_to_file(path, contents[i]) 
    